# Finetuning demo for Whisper
codes adapted from Whisper community [here](https://colab.research.google.com/drive/1P4ClLkPmfsaKn2tBbRp0nVjGMRKR-EWz) and [here](https://huggingface.co/blog/fine-tune-whisper#prepare-environment)

Whisper_base finetuned on Chinese only.

**Content:**

- Preparation
    - Prepare environment
    - Prepare commonvoice/fleurs dataset
    - Prepare Whisper pretrained processor
    - Preprocess dataset with Whisper encoder (saved and can be reloaded)
    - Prepare evaluation metrics (**Start from here with pre-loaded dataset and whisper processor**)
- Finetune
    - Load pretrained Whisper for conditional generation
    - Train Whisper on prepared dataset

## Prepare Env

In [1]:
!pwd
!conda env list

/home/sivan/asr/whisper_finetune
# conda environments:
#
base                     /opt/conda
pytorch_env           *  /opt/conda/envs/pytorch_env



In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov  6 23:57:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 495.46       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Error: must run as root
Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [4]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-848rkjj1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-848rkjj1
  Resolved https://github.com/huggingface/transformers to commit 504db92e7da010070c36e185332420a1d52c12b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/sivan/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Load dataset from Common Voice

downloading zh-CN set took about 20 mins stored in /home/sivan/.cache/huggingface/datasets

In [2]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="test", use_auth_token=True)

print(common_voice)

Found cached dataset common_voice_11_0 (/home/sivan/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/zh-CN/11.0.0/8975395f1d50a6b61f707acd3416761702d3b25412f5fb1004e1db51fe7c304a)
Found cached dataset common_voice_11_0 (/home/sivan/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/zh-CN/11.0.0/8975395f1d50a6b61f707acd3416761702d3b25412f5fb1004e1db51fe7c304a)


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 39637
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10581
    })
})


In [3]:
# keep only the audio and transcript
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 39637
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10581
    })
})


## Plan B: try loading fluers instead
taking about 2 mins

In [5]:
from datasets import load_dataset
fleurs_ch = load_dataset("google/fleurs", "cmn_hans_cn")
print(fleurs_ch)

Found cached dataset fleurs (/home/sivan/.cache/huggingface/datasets/google___fleurs/cmn_hans_cn/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 3246
    })
    validation: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 409
    })
    test: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 945
    })
})


In [9]:
print(fleurs_ch["train"][0])

{'id': 1368, 'num_samples': 132480, 'path': '/home/sivan/.cache/huggingface/datasets/downloads/extracted/0453709a21b236bdd65487aa8a2d41cafb9515ca24965f9608ef0983230ce835/cmn_hans_cn/audio/train/5551535928909193992.wav', 'audio': {'path': '5551535928909193992.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00197393,
       -0.00212777, -0.00208348]), 'sampling_rate': 16000}, 'transcription': '如 果 他 们 使 用 航 空 的 方 式 运 输 货 物 在 某 些 航 线 上 可 能 要 花 几 天 的 时 间 才 能 卸 货 和 通 关', 'raw_transcription': '如果他们使用航空的方式运输货物，在某些航线上，可能要花几天的时间才能卸货和通关。', 'gender': 1, 'lang_id': 13, 'language': 'Mandarin Chinese', 'lang_group_id': 6}


In [10]:
fleurs_ch = fleurs_ch.remove_columns(['id', 'num_samples', 'path','gender', 'lang_id', 'language', 'lang_group_id'])
print(fleurs_ch)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription', 'raw_transcription'],
        num_rows: 3246
    })
    validation: Dataset({
        features: ['audio', 'transcription', 'raw_transcription'],
        num_rows: 409
    })
    test: Dataset({
        features: ['audio', 'transcription', 'raw_transcription'],
        num_rows: 945
    })
})


## Load Whisper

In [2]:
# feature extractor
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

2022-11-09 22:24:38.525280: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 22:24:38.717530: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 22:24:39.389573: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-11-09 22:24:39.389659: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [3]:
# tokenizer
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Chinese", task="transcribe")

In [4]:
# processor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Chinese", task="transcribe")

## Prepare Data

In [7]:
print(common_voice["train"][0])

{'audio': {'path': '/home/sivan/.cache/huggingface/datasets/downloads/extracted/a431e9b847c1f8a129320d049fd36b92f5f9b5f5cc3afb46b78b1203b26a4d25/cv-corpus-11.0-2022-09-21/zh-CN/clips/common_voice_zh-CN_18531536.mp3', 'array': array([0.        , 0.        , 0.        , ..., 0.00105894, 0.00092286,
       0.00065482], dtype=float32), 'sampling_rate': 48000}, 'sentence': '汉元鼎六年，武帝平定南越国，南越之地重新划郡，番禺仍为南海郡治。'}


In [8]:
# from datasets import Audio
# resampling from 48kHz to 16kHz
# common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [14]:
print(fleurs_ch["train"][0])

{'audio': {'path': '5551535928909193992.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00197393,
       -0.00212777, -0.00208348]), 'sampling_rate': 16000}, 'transcription': '如 果 他 们 使 用 航 空 的 方 式 运 输 货 物 在 某 些 航 线 上 可 能 要 花 几 天 的 时 间 才 能 卸 货 和 通 关', 'raw_transcription': '如果他们使用航空的方式运输货物，在某些航线上，可能要花几天的时间才能卸货和通关。'}


In [18]:
# batch prepare
def prepare_dataset(batch):
    # load resampled audio data
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["raw_transcription"]).input_ids
    return batch

In [21]:
# Will take ~40 mins on 16 processes for common voice (and always crash OOM)
# Fleurs takes 3/0.5/1 min for 812/102/237 splits for 4 processes
fleurs_ch = fleurs_ch.map(prepare_dataset, remove_columns=fleurs_ch.column_names["train"], num_proc=4)

#0:   0%|          | 0/812 [00:00<?, ?ex/s]

#1:   0%|          | 0/812 [00:00<?, ?ex/s]

#3:   0%|          | 0/811 [00:00<?, ?ex/s]

#2:   0%|          | 0/811 [00:00<?, ?ex/s]

#0:   0%|          | 0/103 [00:00<?, ?ex/s]

...

#3:   0%|          | 0/102 [00:00<?, ?ex/s]

#0:   0%|          | 0/237 [00:00<?, ?ex/s]

#1:   0%|          | 0/236 [00:00<?, ?ex/s]

#3:   0%|          | 0/236 [00:00<?, ?ex/s]

#2:   0%|          | 0/236 [00:00<?, ?ex/s]

In [22]:
print(fleurs_ch)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3246
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 409
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 945
    })
})


In [24]:
# save processed data to bucket
fleurs_ch.save_to_disk('/home/sivan/datasets/fl_ch_features')

In [25]:
# uploading takes ~30 secs for 4G
!gsutil -m cp -n -r /home/sivan/datasets/fl_ch_features gs://capstone_datasets/fleurs/preprocess/fl_ch_features

Copying file:///home/sivan/datasets/fl_ch_features/dataset_dict.json [Content-Type=application/json]...
Copying file:///home/sivan/datasets/fl_ch_features/test/state.json [Content-Type=application/json]...
Copying file:///home/sivan/datasets/fl_ch_features/test/dataset.arrow [Content-Type=application/octet-stream]...
Copying file:///home/sivan/datasets/fl_ch_features/test/dataset_info.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled 

## Load data from disk

In [5]:
from datasets import load_dataset, load_from_disk
fleurs_ch = load_from_disk('/home/sivan/datasets/fl_ch_features')
print(fleurs_ch)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3246
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 409
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 945
    })
})


In [6]:
# data collator
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [7]:
# initialize collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## Prepare eval

In [8]:
import evaluate

metric = evaluate.load("wer")

In [9]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Finetuning

### Load pretrained Whisper

In [10]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

In [11]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Training config

In [12]:
# reduce per_device_train_batch_size if cuda OOM
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/home/sivan/whisper_base_fl_ch",
    per_device_train_batch_size=16, # originally 16
    gradient_accumulation_steps=1,  # originally 1, increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=False, # original True
    group_by_length=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    disable_tqdm=True,
)

In [13]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=fleurs_ch["train"],
    eval_dataset=fleurs_ch["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer.train()

/home/sivan/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3246
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4000
  Number of trainable parameters = 72593920
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cac

{'loss': 1.7758, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.12}


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

{'loss': 1.4472, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.25}


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

...

Saving model checkpoint to /home/sivan/whisper_base_fl_ch/checkpoint-4000
Configuration saved in /home/sivan/whisper_base_fl_ch/checkpoint-4000/config.json


{'eval_loss': 0.6526612043380737, 'eval_wer': 94.33647973348141, 'eval_runtime': 260.7581, 'eval_samples_per_second': 3.624, 'eval_steps_per_second': 0.456, 'epoch': 19.7}


Model weights saved in /home/sivan/whisper_base_fl_ch/checkpoint-4000/pytorch_model.bin
Feature extractor saved in /home/sivan/whisper_base_fl_ch/checkpoint-4000/preprocessor_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /home/sivan/whisper_base_fl_ch/checkpoint-3000 (score: 93.11493614658522).


{'train_runtime': 19400.6337, 'train_samples_per_second': 3.299, 'train_steps_per_second': 0.206, 'train_loss': 0.10508605905435979, 'epoch': 19.7}


TrainOutput(global_step=4000, training_loss=0.10508605905435979, metrics={'train_runtime': 19400.6337, 'train_samples_per_second': 3.299, 'train_steps_per_second': 0.206, 'train_loss': 0.10508605905435979, 'epoch': 19.7})

In [15]:
import datetime
now = datetime.datetime.now()
print(now)
# training time about 6.5 hours

2022-11-10 03:53:03.209681
